## Lie transformations and symplectic integration

### Imports and setup

In [ ]:
# Standard imports
import sympy as sym
import numpy as np
import IPython

# Import the lie_transformations module
import lie_transformations as lt


In [ ]:
# Define variables used as symbols
i, x, y, px, py = sym.symbols("i, x, y, p_x, p_y", real=True)
delta = sym.symbols("\delta", real=True)
L = sym.symbols("L", real=True)
X = sym.Array([x, y])
P = sym.Array([px, py])


### Generate a map and check the expression

In [ ]:
# Generate hamiltionian for a quadrupole
H = lt.generate_hamiltonian(1, delta, X, P, approx=False)
H


In [ ]:
x_new, px_new, y_new, py_new = lt.lie_transformation_hamiltonian_4D(
    X, P, H, L, order=3, verbose=True
)
x_new


### Generate and plot a particle distributions

In [ ]:
# Generate a particle distribution and get corresponding parameters of the distribution
std_x = 10**-3
std_px = 5 * 10**-4
max_std_x = np.max([std_x, std_px])
array_x, array_px = lt.generate_particle_distribution(std_x, std_px)
emittance_x, alpha_x, beta_x, gamma_x = lt.get_twiss_and_emittance(array_x, array_px)
l_colors_x = lt.get_color_according_to_PCA_projection(array_x, array_px)

# Same with y coordinate (used later)
std_y = 10**-3
std_py = 10**-4
max_std_y = np.max([std_y, std_py])
array_y, array_py = lt.generate_particle_distribution(std_y, std_py)
emittance_y, alpha_y, beta_y, gamma_y = lt.get_twiss_and_emittance(array_y, array_py)
l_colors_y = lt.get_color_according_to_PCA_projection(array_y, array_py)


In [ ]:
lt.plot_distribution(
    array_x,
    array_px,
    l_colors_x,
    max_std_x,
    label_x=r"$x$",
    label_y=r"$p_x$",
    title="Initial particle distribution",
)


### Simulate the transfer of a beam in a quadrupole with truncated Lie transform

In [ ]:
# Plot the result as an interactive plot
gif_path = "gifs/1.gif"
box = lt.plot_interactive_distribution_quadrupole(
    X, P,
    array_x,
    array_px,
    l_colors_x,
    max_std_x,
    label_x=r"$x$",
    label_y=r"$p_x$",
    title="Particle distribution after going through quadrupole",
    max_order_lie_transform=6,
    integrator="truncated_map",
    animation=True,
    gif_path=gif_path,
)
#display(box)
display(IPython.display.Image(data=open(gif_path,'rb').read(), format='gif'))


### Now try again with symplectic integration

In [ ]:
# Plot the result as an interactive plot
gif_path = "gifs/2.gif"
box = lt.plot_interactive_distribution_quadrupole(
    X,
    P,
    array_x,
    array_px,
    l_colors_x,
    max_std_x,
    label_x=r"$x$",
    label_y=r"$p_x$",
    title="Particle distribution after going through quadrupole",
    max_order_lie_transform=10,
    order_symplectic_integrator=3,
    integrator="symplectic_integrator",
    animation=False,
    gif_path=gif_path,
)
# display(IPython.display.Image(data=open(gif_path,'rb').read(), format='gif'))
display(box)


### Compare the phase space in x and y when going through a quadrupole, depending if symplectic integrator or not

In [ ]:
# Plot the result as an interactive plot
gif_path = "gifs/3.gif"
box = lt.plot_interactive_distribution_x_y(
    X,
    P,
    array_x,
    array_px,
    array_y,
    array_py,
    l_colors_x,
    l_colors_y,
    max_std_x,
    max_std_y,
    label_x_1=r"$x$",
    label_y_1=r"$p_x$",
    label_x_2=r"$y$",
    label_y_2=r"$p_y$",
    title="Particle distribution after going through quadrupole",
    max_order_lie_transform=4,
    order_symplectic_integrator=3,
    magnet_order=1,
    exact_map_quadrupole=True,
    animation=False,
    gif_path=gif_path,
)
display(box)


### Check that total emittance is constant

In [ ]:
(
    l_emittance_x_truncated,
    l_emittance_y_truncated,
    l_emittance_x_symplectic,
    l_emittance_y_symplectic,
    l_emittance_x_exact,
    l_emittance_y_exact,
    l_strength,
) = lt.check_emittance_conservation_quadrupole(
    X,
    P,
    array_x,
    array_px,
    array_y,
    array_py,
    max_order_lie_transform=6,
    order_symplectic_integrator=4,
)

lt.plot_emittance_conservation(
    l_emittance_x_truncated,
    l_emittance_y_truncated,
    l_emittance_x_symplectic,
    l_emittance_y_symplectic,
    l_emittance_x_exact,
    l_emittance_y_exact,
    l_strength,
)


### Same plot as before with a nonlinear magnet (sextupole)

In [ ]:
# Plot the result as an interactive plot
gif_path = "gifs/4.gif"
box = lt.plot_interactive_distribution_x_y(
    X, P,
    array_x,
    array_px,
    array_y,
    array_py,
    l_colors_x,
    l_colors_y,
    max_std_x,
    max_std_y,
    label_x_1=r"$x$",
    label_y_1=r"$p_x$",
    label_x_2=r"$y$",
    label_y_2=r"$p_y$",
    title="Particle distribution after going through sextupole",
    max_order_lie_transform=6,
    order_symplectic_integrator=4,
    magnet_order = 2,
    k_min = 0.01,
    k_max = 1000.,
    exact_map_quadrupole = False,
    animation = False,
    gif_path = gif_path,
    duration = 5,
)
display(box)